# 简介

Numba通过及时编译机制（JIT,just in time）优化Python代码，同时支持CPU和GPU的优化，并且可以和Numpy集成

## 优势

> 简单，往往只要1行代码就有惊喜

> 对循环（loop）有奇效，而往往在科学计算中限制python速度的就是loop

> 兼容常用的科学计算包，如numpy、cmath等

> 可以创建ufunc

> 会自动调整精度，保证准确性

# 实际使用

## @nb.jit()装饰器

In [1]:
import numba as nb

In [2]:
@nb.jit()
def nb_sum(a):
    Sum = 0
    for i in range(len(a)):
        Sum += a[i]
    return Sum

# 没用numba加速的求和函数
def py_sum(a):
    Sum = 0
    for i in range(len(a)):
        Sum += a[i]
    return Sum

### 明显加快loop速度

In [5]:
import numpy as np
a = np.linspace(0,100,100) # 创建一个长度为100的数组

%timeit np.sum(a) # numpy自带的求和函数
%timeit sum(a) # python自带的求和函数
%timeit nb_sum(a) # numba加速的求和函数
%timeit py_sum(a) # 没加速的求和函数

3.37 µs ± 207 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
18.1 µs ± 456 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
407 ns ± 18.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
22.5 µs ± 1.37 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### 对numpy自带的函数也加速良好
numpy的加速只适用于numpy自带的函数

In [6]:
a = np.linspace(0,100,10**6) # 创建一个长度为100万的数组

%timeit np.sum(a) # numpy自带的求和函数
%timeit sum(a) # python自带的求和函数
%timeit nb_sum(a) # numba加速的求和函数
%timeit py_sum(a) # 没加速的求和函数

985 µs ± 67.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
150 ms ± 2.94 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
1.11 ms ± 15.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
220 ms ± 13.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## @nb.vectorize()矢量化

### 创建ufunc
numpy的ufunc (universal functions)，可以让一个函数同时处理很多数据

当遇到numpy没有的数学函数时（比如sech），用numba矢量化不失为一个好的选择

In [7]:
from math import sin

@nb.vectorize()
def nb_vec_sin(a):
    return sin(a)

In [8]:
%timeit nb_vec_sin(a)
%timeit np.sin(a)

9.78 ms ± 190 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
10.3 ms ± 299 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## 会自动调整精度，保证准确性

In [10]:
a = np.arange(10**6)

nb_sum(a)

499999500000

In [11]:
np.sum(a)

1783293664

其实在运行的时候，应该报错“RuntimeWarning: overflow encountered in long_scalars”，但奇怪的是np.sum并没有报错

numba给出的结果是int64，其它给出的是int32

## 其它加速类型

@nb.jit(nopython=True,fastmath=True) 牺牲一丢丢数学精度来提高速度

@nb.jit(nopython=True,parallel=True) 自动进行并行计算

# 写CUDA

In [ ]:
import numpy as np 
from timeit import default_timer as timer
from numba import vectorize

@vectorize(["float32(float32, float32)"], target='cuda')
def vectorAdd(a, b):
    return a + b

def main():
    N = 320000000

    A = np.ones(N, dtype=np.float32 )
    B = np.ones(N, dtype=np.float32 )
    C = np.zeros(N, dtype=np.float32 )

    start = timer()
    C = vectorAdd(A, B)
    vectorAdd_time = timer() - start

    print("c[:5] = " + str(C[:5]))
    print("c[-5:] = " + str(C[-5:]))

    print("vectorAdd took %f seconds " % vectorAdd_time)

if __name__ == '__main__':
    main()